In [105]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [131]:
data = pd.read_csv('/content/items.csv')

In [133]:
data.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15963 entries, 0 to 15962
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       15963 non-null  int64  
 1   content_type  15963 non-null  object 
 2   title         15963 non-null  object 
 3   title_orig    11218 non-null  object 
 4   release_year  15865 non-null  float64
 5   genres        15963 non-null  object 
 6   countries     15926 non-null  object 
 7   for_kids      566 non-null    float64
 8   age_rating    15961 non-null  float64
 9   studios       1065 non-null   object 
 10  directors     14454 non-null  object 
 11  actors        13344 non-null  object 
 12  description   15963 non-null  object 
 13  keywords      15540 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+ MB


In [107]:
# Предобработка данных: замена пропущенных значений в столбце 'description' на пустые строки
data['description'].fillna('', inplace=True)

In [108]:
# Извлечение признаков с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

In [109]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test = train_test_split(tfidf_matrix, test_size=0.2, random_state=42)

In [110]:
# Обучение модели
model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine')
model.fit(X_train)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [114]:
# Протестировать модель и сформировать рекомендации для каждого пользователя
recommendations = {}
for i in range(X_test.shape[0]):
    query = X_test[i]
    _, indices = model.kneighbors(query)
    item_id = data.iloc[i]['item_id']
    recommendations[item_id] = indices

In [123]:
# Вывод рекомендаций для каждого пользователя
for item_id, indices in recommendations.items():
    print(f"Recommendations for user {item_id}:")
    for index in indices:
        print(data.iloc[index]['title'])

Выходные данные были обрезаны до нескольких последних строк (5000).
5198                         Маменькин сынок
8515                                 Дилер 3
2387                             Потрошители
6918                         10 минут спустя
7077     Даже не думай 2: Тень независимости
1427                          Детки напрокат
11410                        Интимная студия
Name: title, dtype: object
Recommendations for user 8737:
5742                                   Беспокойный участок
1909     Волшебное зеркало, или Двойные неприятности (с...
1515                                         Убийство кота
11998                                            Рокировка
8903            Белоснежка: Месть гномов (жестовым языком)
5247                                      Женщина в беде 2
5853                                       Принцессы Мафии
12366                               Говорящий Том и Друзья
3423                                  Мой любимый динозавр
627                         